<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

This exercise covers two aspects:
* In tasks 1-6 you will implement mechanisms that allow training deeper models (better initialization, batch normalization). Note that for dropout and batch norm you are expected to implement it yourself without relying on ready-made components from Pytorch.
* In task 7 you will implement a convnet using [conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html).


Tasks:
1. Check that the given implementation reaches 95% test accuracy for
   architecture input-64-64-10 in a few thousand batches.
2. Improve initialization and check that the network learns much faster
   and reaches over 97% test accuracy. A good basic initialization scheme is so-called Glorot initialization. For a set of weights going from a layer with $n_{in}$ neurons to a layer with $n_{out}$ neurons, it samples each weight from normal distribution with $0$ mean and standard deviation of $\sqrt{\frac{2}{n_{in}+n_{out}}}$.
3. Check, that with proper initialization we can train architecture
   input-64-64-64-64-64-10, while with bad initialization it does
   not even get off the ground.
4. Add dropout implemented in pytorch (but without using torch.nn.Dropout)
5. Check that with 10 hidden layers (64 units each) even with proper
    initialization the network has a hard time to start learning.
6. Implement batch normalization (use train mode also for testing - it should perform well enough):
    * compute batch mean and variance
    * add new variables beta and gamma
    * check that the networks learns much faster for 5 layers
    * check that the network learns even for 10 hidden layers.
7. So far we worked with a fully connected network. Design and implement in pytorch (by using pytorch functions) a simple convolutional network and achieve 99% test accuracy. The architecture is up to you, but even a few convolutional layers should be enough.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn import init
import torchvision
import torchvision.transforms as transforms

In [ ]:
class Linear(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        self.bias = Parameter(torch.Tensor(out_features))
        self.glorot_initizalization()

    def reset_parameters(self):
        self.weight.data.normal_(mean=0,std=0.25)
        init.zeros_(self.bias)

    def glorot_initizalization(self):
        self.weight.data.normal_(mean=0,std=(2 / (self.in_features + self.out_features)) ** 0.5)
        init.zeros_(self.bias)

    def forward(self, x):
        r = x.matmul(self.weight.t())
        r += self.bias
        return r

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        # fully connected layer, output 10 classes
        self.out = Linear(32 * 8 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.fc1 = Linear(784, 64)
        # self.fc2 = Linear(64, 64)
        # self.fc3 = Linear(64, 10)
        self.fc1 = Linear(784, 64)
        self.fc2 = Linear(64, 64)
        self.fc3 = Linear(64, 64)
        self.fc4 = Linear(64, 64)
        self.fc5 = Linear(64, 64)
        self.fc6 = Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x


In [ ]:
class MnistTrainer(object):
    def __init__(self, batch_size):
        transform = transforms.Compose(
                [transforms.ToTensor()])
        self.trainset = torchvision.datasets.MNIST(
            root='./data',
            download=True,
            train=True,
            transform=transform)
        self.trainloader = torch.utils.data.DataLoader(
            self.trainset, batch_size=batch_size, shuffle=True, num_workers=2)

        self.testset = torchvision.datasets.MNIST(
            root='./data',
            train=False,
            download=True, transform=transform)
        self.testloader = torch.utils.data.DataLoader(
            self.testset, batch_size=1, shuffle=False, num_workers=2)

    def train(self):
        net = ConvNet()

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)

        for epoch in range(20):
            running_loss = 0.0
            for i, data in enumerate(self.trainloader, 0):
                inputs, labels = data
                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 100 == 99:
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for data in self.testloader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the {} test images: {} %'.format(
                total, 100 * correct / total))

In [ ]:
trainer = MnistTrainer(batch_size=128)
trainer.train()

[1,   100] loss: 0.528
[1,   200] loss: 0.109
[1,   300] loss: 0.082
[1,   400] loss: 0.068
Accuracy of the network on the 10000 test images: 98.32 %
[2,   100] loss: 0.048
[2,   200] loss: 0.047
[2,   300] loss: 0.047
[2,   400] loss: 0.052
Accuracy of the network on the 10000 test images: 98.75 %
[3,   100] loss: 0.033
[3,   200] loss: 0.033
[3,   300] loss: 0.043
[3,   400] loss: 0.032
Accuracy of the network on the 10000 test images: 98.82 %
[4,   100] loss: 0.023
[4,   200] loss: 0.034
[4,   300] loss: 0.026
[4,   400] loss: 0.028
Accuracy of the network on the 10000 test images: 98.8 %
[5,   100] loss: 0.020
[5,   200] loss: 0.018
[5,   300] loss: 0.029
[5,   400] loss: 0.025
Accuracy of the network on the 10000 test images: 98.93 %
[6,   100] loss: 0.016
[6,   200] loss: 0.018
[6,   300] loss: 0.016
[6,   400] loss: 0.022
Accuracy of the network on the 10000 test images: 99.01 %
[7,   100] loss: 0.017
[7,   200] loss: 0.014
[7,   300] loss: 0.015
[7,   400] loss: 0.015
Accuracy 

KeyboardInterrupt: ignored

In [ ]:
trainer = MnistTrainer(batch_size=128)
trainer.train()

[1,   100] loss: 0.748
[1,   200] loss: 0.152
[1,   300] loss: 0.114
[1,   400] loss: 0.098
Accuracy of the network on the 10000 test images: 98.0 %
[2,   100] loss: 0.072
[2,   200] loss: 0.072
[2,   300] loss: 0.062
[2,   400] loss: 0.063
Accuracy of the network on the 10000 test images: 97.89 %
[3,   100] loss: 0.053
[3,   200] loss: 0.045
[3,   300] loss: 0.049
[3,   400] loss: 0.049
Accuracy of the network on the 10000 test images: 98.55 %
[4,   100] loss: 0.042
[4,   200] loss: 0.041
[4,   300] loss: 0.041
[4,   400] loss: 0.040
Accuracy of the network on the 10000 test images: 98.71 %
[5,   100] loss: 0.035
[5,   200] loss: 0.035
[5,   300] loss: 0.031
[5,   400] loss: 0.036
Accuracy of the network on the 10000 test images: 98.68 %
[6,   100] loss: 0.031
[6,   200] loss: 0.030
[6,   300] loss: 0.031
[6,   400] loss: 0.026
Accuracy of the network on the 10000 test images: 98.83 %
[7,   100] loss: 0.027
[7,   200] loss: 0.021
[7,   300] loss: 0.029
[7,   400] loss: 0.033
Accuracy 